# Database


In [2]:
! pip install mysql.connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:01m


In [8]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "admin",
    database="movies_db"

)

cursor = dbconnection.cursor()

print(dbconnection.database)

cursor.close()
dbconnection.close()

movies_db


## CREATE OPERATIONS

In [9]:
import mysql.connector 

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "admin",
    database="movies_db"

)

cursor = dbconnection.cursor()

query = "INSERT INTO actors(name) VALUES('Kevin Hart')"

cursor.execute(query)
dbconnection.commit()

cursor.close()
dbconnection.close()

In [10]:
import mysql.connector 

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "admin",
    database="movies_db"

)

cursor = dbconnection.cursor()

query = "INSERT INTO actors(name) VALUES(%s)"

data = [["Leonardo dicarpatio"],["Samuel"],["al pacino"],["jolie"],["brad pit"]]

cursor.executemany(query,data)
dbconnection.commit()

cursor.close()
dbconnection.close()

In [12]:
import mysql.connector

# establish connection
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="admin",
  database="movies_db"
)

# prepare query
query = "INSERT INTO movies (title, synopsis) VALUES (%s, %s)"

# list of movies to insert
movies = [
  ("Inception", "A thief is hired to plant an idea in the mind of a corporate executive."),
  ("Blade Runner", "A detective hunts down rogue replicants in a dystopian future."),
  ("Star Wars", "A farm boy joins a rebellion to overthrow an evil empire."),
  ("Jurassic Park", "Scientists create a theme park with cloned dinosaurs, which quickly goes awry.")
]

# insert each movie
cursor = db.cursor()
for movie in movies:
  cursor.execute(query, movie)

# commit changes and close connection
db.commit()
db.close()

In [21]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "admin",
    database="movies_db"
)

cursor = dbconnection.cursor()

query = """SELECT movies.title, actors.name FROM movies
           inner JOIN movie_actor ON movies.id = movie_actor.movie_id
           inner JOIN actors ON movie_actor.actor_id = actors.id
           """


cursor.execute(query)
data = cursor.fetchall()


for row in data:
    print(row)

('Everything Everywhere All at Once', 'Michelle Yeoh')
('Everything Everywhere All at Once', 'Stephan James')
('Everything Everywhere All at Once', 'Jamie Lee Curtis')
('Top Gun: Maverick', 'Tom Cruise')


### check slides for querrying with sql

In [24]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "admin",
    database="movies_db"
)

cursor = dbconnection.cursor()

#query = "INSERT INTO movies(title) VALUES('Jumanji')"

#cursor.execute(query)
#dbconnection.commit()

query = "UPDATE movies SET synopsis = 'board games' where title like 'Jumanji'"

cursor.execute(query)

dbconnection.commit()

cursor.close()
dbconnection.close()

## Scrape to DB exersice 

https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films

put all movies in a db

In [32]:
import requests
from bs4 import BeautifulSoup

url ="https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films"

resp = requests.get(url)
soup = BeautifulSoup(resp.text)

#open db

import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "admin",
    database="movies_db"
)
cursor = dbconnection.cursor()

table = soup.find("table")
rows = table.find_all("tr")

for row in rows:
    td = row.find_all("td")

    if len(td) > 0:
        title = td[0].get_text().strip().replace("'","")
        year = td[1].get_text().strip().replace("'","")
        awards = td[2].get_text().strip().replace("'","")
        nom = td[3].get_text().strip().replace("'","")


        #check if the movie is already in the db

        query = "SELECT * from movies where title like '" + title + "'"

       # print(query)
        cursor.execute(query)
        data = cursor.fetchall()

        if len(data) == 0:
            query = "INSERT into movies(title) VALUES('" + title + "')"
            cursor.execute(query)
            dbconnection.commit()
            movie_id = cursor.lastrowid

            query = "Insert into awards(movie_id,awardshow_id,year,awards,nominations) values(%s,%s,%s,%s,%s)"

            values = (movie_id,1,year,awards,nom)

            cursor.execute(query,values)
            dbconnection.commit()


cursor.close()
dbconnection.close()

DatabaseError: 1265 (01000): Data truncated for column 'year' at row 1